In [ ]:
# Mechanism to mock test submits in MongoDB with random dates.
# the term `test_sic_collection` refers to collection track search and has been refactored to sic collection.
from bson import ObjectId
from config.database import db_UJ        # UJ mongoDB
from datetime import datetime, timedelta
from pymongo import UpdateOne
import random

class MockTestSubmits:
    def __init__(self):
        self.test_sic_collection = db_UJ['test_st']

    def random_date_gen(self, simplify=False, total_dates=5):
        def df(dt):
            return dt.strftime("%m-%d")

        now = datetime.now()  # Get current date and time
        dates = []

        # Generate dates for the last 3 days (today and the two previous days)
        for i in range(3):
            date = now - timedelta(days=i)
            dates.append(date)

        # Randomly select 5 items from these dates (with replacement)
        random_items = random.choices(dates, k=random.randint(1, total_dates))
        # If simplify is True, format the dates
        if simplify:
            # Format the dates using your df function
            formatted_items = [df(item) for item in random_items]
            return formatted_items    
        else:
            return random_items
        # Format the dates using your df function
        # formatted_items = [df(item) for item in random_items]
        # return formatted_items

    def update_operations(self,mdb_set):
        test_sic_collection = db_UJ['test_st']   # create/get a collection
        doc_ids = list(test_sic_collection.find({},{"_id":1}))       # Get limited test document IDs
        update_operation = []
        for i in doc_ids:
            juice = {'submits':self.random_date_gen(simplify=False, total_dates=15)}
            # Use this to remove all the submits fields
            # Use this to simulate setting the new submits field with mock timestamps
            set_ops = {'$set': juice} if mdb_set else {'$unset': {'submits': ''}}
            update_operation.append(
                UpdateOne(
                {"_id": ObjectId(i['_id'])},
                set_ops
            ))
        return update_operation

    def update_submits(self, mdb_set):
        self.test_sic_collection.bulk_write(self.update_operations(mdb_set=mdb_set))

    def check_transformed_submits(self):
        original = list(self.test_sic_collection.find({'submits': {'$exists': True}},{"_id":0,"ph":0,"r_id":0}))
        transformed = [{v1: v2} for d in original for v1, v2 in zip(d.values(), list(d.values())[1:])]
        return transformed

mts = MockTestSubmits()
# mts.update_operations(mdb_set=False)
# mts.update_submits(mdb_set=False)

In [23]:
# simulating all combinations of searches on frontend -> backend
import itertools
import string

# Generate all combinations of length 2. numbers are not considered.
test_items = 5
alphabets = string.ascii_lowercase[:test_items]
combinations = [''.join(c) for c in itertools.combinations(alphabets, 2)]
permutations = [''.join(p) for p in itertools.permutations(alphabets, 2)]

# Combinations would miss some items for example dl is once considered but ld wont be considered
    # in permutations however, dl and ld both is considered.
    # Another example would be aa  -- `aa` is considered in permutations but is not considered in combinations
    # So use permutations instead of combinations.
print('combs',combinations)
print('permuts',permutations)
print(f"Total combinations: {len(combinations)}", f"Total permut: {len(permutations)}")

combs ['ab', 'ac', 'ad', 'ae', 'bc', 'bd', 'be', 'cd', 'ce', 'de']
permuts ['ab', 'ac', 'ad', 'ae', 'ba', 'bc', 'bd', 'be', 'ca', 'cb', 'cd', 'ce', 'da', 'db', 'dc', 'de', 'ea', 'eb', 'ec', 'ed']
Total combinations: 10 Total permut: 20


In [1]:
from config.database import db_UJ        # UJ mongoDB
# db_UJ.list_collection_names()
# og_collection = db_UJ['search_index'] # create/get a collection. if it doesn't exist then make one.
test_collection = db_UJ['test_st'] # create/get a collection. if it doesn't exist then make one.
find_crit = {}
return_crit = {'submits':0}
c = list(test_collection.find(find_crit,return_crit).sort('ph',-1))


In [45]:
# c

In [1]:
# # Copy a collection to a new onne
# new_collection = 'test_st'  # Replace 'target_collection_name' with your desired target collection name

# # Copy the collection
# og_collection.aggregate([{'$out': 'test_st'}])


In [ ]:
cts_returns =  list(mts.cts.aggregate([
        { "$match": { "submits": { "$exists": True} } },
        { "$unwind": "$submits" },
        { "$addFields": { "timestamp": "$submits" } },
        { "$unset": ["_id", "r_id", "ph", "submits"] }
    ]))

In [ ]:
list(mts.cts.find({"submits": {"$exists": True}}))

In [2]:
from config.database import db_UJ        # UJ mongoDB
# Use search interface to find sic items that match query given search index collection docs as 'sic'
from routes.root.search.search_interface import SearchInterface
from routes.root.search.fuzz_find import fuzz_find
from routes.root.search.query_classifier import QueryClassifier

search_index_collection = db_UJ['search_index']

count_crit = {'ph':{"$exists":True}}       # return ones with popularity hits
# return_crit = {'ph':0, 'submits':0}       # return only...
return_crit = {'submits':0}       # return only...
sic = list(search_index_collection.find(count_crit,return_crit).sort('ph',-1))     # Reverse sort


C:\Users\ujasv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [25]:
qc = QueryClassifier(icao_file_path="unique_icao.pkl")
limit = 500
query='denv'
ff = SearchInterface()
formatted_suggestions = ff.search_suggestion_frontned_format(c_docs=sic)
sti_items_match_w_query = fuzz_find(query=query, data=formatted_suggestions, qc=qc, limit=limit)



In [ ]:
# (sti_items_match_w_query)
# len(formatted_suggestions)


[]

In [ ]:
from routes.route import get_search_suggestions


query='den'
x = await get_search_suggestions(email='Anonymous',query=query)
# x